# Task 1

# Generate training data

### Initialization

In [1]:
%matplotlib inline
import numpy as np
import math
import random
import pyJHTDB
import h5py
import time as tt
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import filters
import skimage.morphology as morphology
from skimage.morphology import square
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from matplotlib import cm
import scipy.io as sio
import time
import csv
import matplotlib.animation as animation
# load shared library
lTDB = pyJHTDB.libJHTDB()
#initialize webservices
lTDB.initialize()

#Add token
auth_token  = "edu.jhu.burgers-2e898d7f"
lTDB.add_token(auth_token)

dataset = 'transition_bl'

spatialInterp  = 0  # no spatial interpolation
temporalInterp = 0  # no time interpolation
FD4NoInt       = 40 # 4th-order FD, no spatial interpolation

### Database parameters

In [2]:
# Database domain size and number of grid points
x_min =   30.2185
x_max = 1000.0650
y_min =    0.0036
y_max =   26.4880
z_min =    0.0000
z_max =  240.0000
d99i  =    0.9648
d99f  =   15.0433

nx = 3320
ny =  224
nz = 2048

# Database time duration
Ti = 0
Tf = Ti + 1175
dt = 0.25

### Define domain size

In [3]:
# # Create surface
# nix = round(nx / 10)
# niz = round(nz / 10)
# niy = round(ny / 1)
# x = np.linspace(x_min, x_max, nix)
# z = np.linspace(z_min, z_max, niz)
# y = np.linspace(y_min, y_max, niy); #d99i

# [X,Z] = np.meshgrid(x,z)
# print(niz,nix,niy,nix*niy*niz)

### getVelocity  &  getVelocityGradient

In [4]:
# # Get the velocity at each point

# time = 1000
# ui=np.zeros((nix,niz,niy,3))
# duidxj=np.zeros((nix,niz,niy,9))

# for j in range(200,224):
#     print('Read y:',j,y[j])
#     start = tt.time()
#     points  = np.zeros((nix,niz,3))
#     points[:,:,0] = X.transpose()
#     points[:,:,1] = y[j]
#     points[:,:,2] = Z.transpose()

#     # 3D array with single precision values
#     points = np.array(points,dtype='float32')

#     print('Requesting velocity at {0} points...'.format(nix*niz*1))
#     result = lTDB.getData(time, points, data_set = 'transition_bl',
#             sinterp = spatialInterp, tinterp = temporalInterp,
#             getFunction = 'getVelocity')
#     end = tt.time()
#     print('   '+str(end - start)+' seconds')

#     start = tt.time()
#     print('Requesting velocity gradients at {0} points...'.format(nix*niz*1))
#     result_grad = lTDB.getData(time, points, data_set = 'transition_bl',
#             sinterp = FD4NoInt, tinterp = temporalInterp,
#             getFunction = 'getVelocityGradient')
#     end = tt.time()
#     print('   '+str(end - start)+' seconds')
#     ui[0:nix,0:niz,j,0:3]=result[0:nix,0:niz,0:3]
#     duidxj[0:nix,0:niz,j,0:9]=result_grad[0:nix,0:niz,0:9]
    
    
    
# #Save in matlab format   
# #sio.savemat('data_t1000_415x28x256.mat',{'x':x,'y':y,'z':z,'u_i':result,'duidxj':result_grad})

# #save using hdf5 format
# h5f = h5py.File('data_t1000_332x224x205_224.h5', 'w')
# h5f.create_dataset('x', data=X)
# h5f.create_dataset('y', data=y)
# h5f.create_dataset('z', data=Z)
# h5f.create_dataset('ui', data=ui)
# h5f.create_dataset('duidxj', data=duidxj)
# h5f.close()

### Velocity and velocity gradient input data

In [5]:
data=sio.loadmat('data_t0_224x205x332.mat')
u=data['u'][0::5,0::7,0::7]
v=data['v'][0::5,0::7,0::7]
w=data['w'][0::5,0::7,0::7]
up=data['up'][0::5,0::7,0::7]
vp=data['vp'][0::5,0::7,0::7]
dudx=data['dudx'][0::5,0::7,0::7]
dudy=data['dudy'][0::5,0::7,0::7]
dudz=data['dudz'][0::5,0::7,0::7]

dvdx=data['dvdx'][0::5,0::7,0::7]
dvdy=data['dvdy'][0::5,0::7,0::7]
dvdz=data['dvdz'][0::5,0::7,0::7]

dwdx=data['dwdx'][0::5,0::7,0::7]
dwdy=data['dwdy'][0::5,0::7,0::7]
dwdz=data['dwdz'][0::5,0::7,0::7]

x=data['x'][0::7]
y=data['y'][0::5]
z=data['z'][0::7]
[X,Y,Z]=np.meshgrid(x,y,z)
nix=x.shape[0]
niy=y.shape[0]
niz=z.shape[0]

In [6]:
u.shape,niy,niz,nix

((45, 30, 48), 45, 30, 48)

### Input vector definition

In [7]:
input_vector = np.zeros((nix*niz*niy, 16), np.float32)      # Size of input vector (u,v,w,up,vp, du/dxyz,dv/dxyz, dw/dxyz, x,y)
input_vector_zhao = np.zeros((nix*niz*niy, 16), np.float32)      # Size of input vector (u,v,w,up,vp, du/dxyz,dv/dxyz, dw/dxyz, x,y)
std_vector = np.zeros((1, 16), np.float32)      # Size of input vector (u,v,w,up,vp, du/dxyz,dv/dxyz, dw/dxyz, x,y)
input_vector[:,0]=np.abs(u[:,:,:]).flatten()               # u
input_vector[:,1]=np.abs(v[:,:,:]).flatten()               # v
input_vector[:,2]=np.abs(w[:,:,:]).flatten()               # w
input_vector[:,3]=np.abs(up[:,:,:]).flatten()              # up
input_vector[:,4]=np.abs(vp[:,:,:]).flatten()              # vp
input_vector[:,5]=np.abs(dudx[:,:,:]).flatten()            # dudx
input_vector[:,6]=np.abs(dudy[:,:,:]).flatten()            # dudy
input_vector[:,7]=np.abs(dudz[:,:,:]).flatten()            # dudz
input_vector[:,8]=np.abs(dvdx[:,:,:]).flatten()            # dvdx
input_vector[:,9]=np.abs(dvdy[:,:,:]).flatten()            # dvdy
input_vector[:,10]=np.abs(dvdz[:,:,:]).flatten()           # dvdz
input_vector[:,11]=np.abs(dwdx[:,:,:]).flatten()           # dwdx
input_vector[:,12]=np.abs(dwdy[:,:,:]).flatten()           # dwdy
input_vector[:,13]=np.abs(dwdz[:,:,:]).flatten()           # dwdz
input_vector[:,14]=np.abs(X[:,:,:]).flatten()              # x
input_vector[:,15]=np.abs(Y[:,:,:]).flatten()              # y

#Standard deviation and coefficients from Zhao et al 2019
std_vector_zhao=np.array([[0.1312,0.0229,0.0264,0.04,0.0229,0.0243,0.1595,0.0626,
               0.0246,0.0296,0.0385,0.0270,0.0536,0.0306,286.6778,8.2739]])
coeffs_zhao=np.array([0.19, -0.15, -0.16, -0.16, -0.16, -0.17, -0.10, -0.15,
                -0.15, -0.17, -0.16, -0.17, -0.16, -0.17, -0.08, 0.15])
for i in range(input_vector.shape[1]):
    std_vector[0,i]=np.std(input_vector[:,i])
    input_vector_zhao[:,i]=input_vector[:,i]/std_vector_zhao[0,i]  # Normalize by standard devition
    input_vector[:,i]=input_vector[:,i]/std_vector[0,i]  # Normalize by standard devition


In [8]:
#Check if varaiance of input data is 1
for i in range(input_vector.shape[1]):
    print('variance',np.var(input_vector[:,i]),np.var(input_vector_zhao[:,i]))

variance 0.9999999 1.8752782
variance 1.0 0.59614605
variance 0.9999999 0.6413575
variance 1.0 0.65841883
variance 0.9999999 0.58708555
variance 1.0000001 0.931605
variance 1.0000002 2.7819993
variance 1.0000001 0.914742
variance 1.0000001 0.94644463
variance 0.99999994 0.8216153
variance 1.0000001 0.9596662
variance 1.0000001 0.9380482
variance 0.99999994 1.7595795
variance 0.9999998 0.85986406
variance 1.0 0.98235995
variance 0.9999999 0.86879903


### Encode data using Kmeans clustering (SOM)

In [9]:
#Encode data as T or NT using KMeans clustering (Zhao et al, 2018) 
start = tt.time()
print('Machine learning training...')
kmeans = KMeans(n_clusters=2).fit(input_vector)
end = tt.time()
print('   '+str(end - start)+' seconds')

Machine learning training...
   0.5161142349243164 seconds


### Compute weight vector

In [10]:
#Finding co-efficients
#mask=kmeans.labels_.reshape((niz,nix,niy))

a=kmeans.cluster_centers_[0]
b=kmeans.cluster_centers_[1]
c=np.zeros(a.shape[0])
coeffs=np.zeros(a.shape[0])

for i in range(a.shape[0]):
    c[i]=0.5*(a[i]+b[i])
B=0
for i in range(c.shape[0]):
    B=B-c[i]*(b[i]-a[i])
    
#co-efficients
for i in range(c.shape[0]):
    coeffs[i]=(b[i]-a[i])/B
print(coeffs)

[ 0.07276528 -0.10107189 -0.10450955 -0.10480831 -0.10204438 -0.10837419
 -0.02022405 -0.09381035 -0.10640237 -0.11023948 -0.10315286 -0.11028725
 -0.0743276  -0.11148015 -0.00426443  0.08950542]


### Compare weights with Zhao et al, 2019

In [11]:
print(coeffs)
print(coeffs_zhao)

[ 0.07276528 -0.10107189 -0.10450955 -0.10480831 -0.10204438 -0.10837419
 -0.02022405 -0.09381035 -0.10640237 -0.11023948 -0.10315286 -0.11028725
 -0.0743276  -0.11148015 -0.00426443  0.08950542]
[ 0.19 -0.15 -0.16 -0.16 -0.16 -0.17 -0.1  -0.15 -0.15 -0.17 -0.16 -0.17
 -0.16 -0.17 -0.08  0.15]


### Compute the T-NT classification

In [12]:
#T-NT classification of training data
mask=np.zeros((nix*niz*niy),np.int32)
y_true=[]
for i in range(nix*niz*niy):
    p=1
    for k in range(16):
        p=p+coeffs[k]*input_vector[i,k]
    if p<0:
        mask[i]=1#turbulent
        y_true.append("T")
    elif p>0:
        mask[i]=0#laminar
        y_true.append("NT")
        
mask = mask.reshape((niy,niz,nix))
#sio.savemat('mask_t0_SOM_45x30x48.mat',{'mask_t0_SOM':mask,'coeffs_t0_SOM':coeffs,'y_true_t0':y_true})

### Save data in HDF5 file

In [13]:
# h5f = h5py.File('data_t0_TNT_45x30x48.h5', 'w')
# h5f.create_dataset('u',data=input_vector[:,0])           
# h5f.create_dataset('v',data=input_vector[:,1])           
# h5f.create_dataset('w',data=input_vector[:,2])
# h5f.create_dataset('up',data=input_vector[:,3])
# h5f.create_dataset('vp',data=input_vector[:,4])
# h5f.create_dataset('dudx',data=input_vector[:,5])
# h5f.create_dataset('dudy',data=input_vector[:,6])
# h5f.create_dataset('dudz',data=input_vector[:,7])
# h5f.create_dataset('dvdx',data=input_vector[:,8])
# h5f.create_dataset('dvdy',data=input_vector[:,9])
# h5f.create_dataset('dvdz',data=input_vector[:,10])
# h5f.create_dataset('dwdx',data=input_vector[:,11])
# h5f.create_dataset('dwdy',data=input_vector[:,12])
# h5f.create_dataset('dwdz',data=input_vector[:,13])
# h5f.create_dataset('x',data=input_vector[:,14])
# h5f.create_dataset('y',data=input_vector[:,15])
# h5f.create_dataset('Z',data=Z)
# h5f.create_dataset('Y',data=Y)
# h5f.create_dataset('X',data=X)
# y_data = np.array(y_true, dtype=object)
# string_dt = h5py.special_dtype(vlen=str)
# h5f.create_dataset('TNT',data=y_data,dtype=string_dt)
# h5f.close()
